<a href="https://colab.research.google.com/github/SalmanMohammadi/computing_science_diss/blob/master/LSTM_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Hack to allow python to pick up the newly-installed fluidsynth lib.
# import ctypes.util
# def proxy_find_library(lib):
#   if lib == 'fluidsynth':
#     return 'libfluidsynth.so.1'
#   else:
#     return ctypes.util.find_library(lib)

# ctypes.util.find_library = proxy_find_library

# Download Salamander piano SoundFont.
# Samples by Alexander Holm: https://archive.org/details/SalamanderGrandPianoV3
# Converted to sf2 by John Nebauer: https://sites.google.com/site/soundfonts4u
!gsutil -m cp gs://download.magenta.tensorflow.org/soundfonts/Yamaha-C5-Salamander-JNv5.1.sf2 /tmp/

# Setting up Tensorboard

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

Copying gs://download.magenta.tensorflow.org/soundfonts/Yamaha-C5-Salamander-JNv5.1.sf2...
- [1/1 files][591.9 MiB/591.9 MiB] 100% Done  48.3 MiB/s ETA 00:00:00           
Operation completed over 1 objects/591.9 MiB.                                    
--2019-01-16 11:20:15--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.203.66.95, 52.22.145.207, 52.203.53.176, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.203.66.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  3.47MB/s    in 1.5s    

2019-01-16 11:20:18 (3.47 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
tf.reset_default_graph()
!rm -rf /content/log/

In [0]:
!pip install -q magenta\
                music21

In [3]:
# from __future__ import absolute_import
import magenta as mg
import tensorflow as tf
import tensorflow_probability as tfp
# from music21 import converter, instrument, note, chord
# import numpy as np
# import tensorflow.keras as keras

/usr/local/lib/python2.7/dist-packages/scipy/signal/_max_len_seq.py:8: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._max_len_seq_inner import _max_len_seq_inner
/usr/local/lib/python2.7/dist-packages/scipy/signal/_upfirdn.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._upfirdn_apply import _output_len, _apply
/usr/local/lib/python2.7/dist-packages/scipy/signal/spectral.py:10: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._spectral import _lombscargle
/usr/local/lib/python2.7/dist-packages/scipy/signal/_peak_finding.py:13: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._peak_finding_utils import (_argmaxima1d, _select_by_peak_distance,
/usr/local/lib/python2.7/dist-packages/sklearn/linear_model/base.py:35: RuntimeWarning: numpy.dtype size ch

In [4]:
LOG_DIR = '/content/log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://2a14fdff.ngrok.io


In [0]:
!cp -r /content/log/run0/ /saved

In [0]:
!ls /content/log/run0/train

ls: cannot access '/content/log/run0/train': No such file or directory


## Data

In [5]:
# Grab some data
# !git clone https://github.com/Skuldur/Classical-Piano-Composer
# !git clone https://github.com/brannondorsey/midi-rnn
# !wget https://storage.googleapis.com/magentadata/datasets/maestro/v1.0.0/maestro-v1.0.0-midi.zip
# !wget "www.kuhmann.com/Disklavier/Classical-II%20(215)%20MID.zip"
# !wget "www.kuhmann.com/Disklavier/Classical-I%20(001-215)%20MID.zip"
!wget https://storage.googleapis.com/maestro-performance/training_performances.tfrecord

# Organise MIDI files for parsing
# !mkdir data
# !unzip 'Classical-I (001-215) MID.zip' -q -d /content/data/
# !unzip 'Classical-II (215) MID.zip' -d /content/data/
# !unzip 'maestro-v1.0.0-midi.zip' -d  data
# !mv Classical-Piano-Composer/midi_songs/*.mid /content/data/
!mkdir sequences
!mv training_performances.tfrecord sequences/
!ls data

--2019-01-16 11:21:21--  https://storage.googleapis.com/maestro-performance/training_performances.tfrecord
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.97.128, 2404:6800:4008:c00::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.97.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31145373696 (29G) [application/octet-stream]
Saving to: ‘training_performances.tfrecord’

training_performanc 100%[===================>]  29.01G  40.0MB/s    in 18m 56s 

2019-01-16 11:40:18 (26.1 MB/s) - ‘training_performances.tfrecord’ saved [31145373696/31145373696]

ls: cannot access 'data': No such file or directory


In [0]:
# Parse our MIDI files using magenta
!python /usr/local/lib/python2.7/dist-packages/magenta/scripts/convert_dir_to_note_sequences.py \
        --input_dir='data/maestro-v1.0.0' \
        --output_file='tfrecords/notesequences.tfrecord'\
        --recursive

In [0]:
# Augment and slice data for input into our model
!python /usr/local/lib/python2.7/dist-packages/magenta/models/performance_rnn/performance_rnn_create_dataset.py \
        --config=performance_with_dynamics \
        --eval_ratio=0.2 \
        --input=tfrecords/notesequences.tfrecord \
        --output_dir=sequences 

In [0]:
!ls -lh /content/tfrecords

total 25M
-rw-r--r-- 1 root root 25M Jan  7 13:30 notesequences.tfrecord


## Utils

In [0]:
def get_deep_lstm(layers, dropouts):
  cells = [tf.nn.rnn_cell.LSTMCell(size, reuse=tf.AUTO_REUSE) for size in layers]
  cells = [tf.contrib.rnn.DropoutWrapper(cell, prob) for prob, cell in zip(dropouts, cells)]
  return tf.contrib.rnn.MultiRNNCell(cells)
    

## LSTM

In [0]:
from tensorflow.python.util import nest as tf_nest

def build_lstm(config, mode):
  
  def build_graph():

    encoder_decoder = config['encoder_decoder']
    input_size = encoder_decoder.input_size
    num_classes = encoder_decoder.num_classes
    default_event_label = encoder_decoder.default_event_label

    batch_size = 64
    label_shape = []
    learning_rate = config['learning_rate']
    inputs, labels, lengths = None, None, None
    if mode == 'train' or mode == 'eval':
      inputs, labels, lengths = mg.common.get_padded_batch(
                ['/content/sequences/training_performances.tfrecord'], batch_size, input_size,
                label_shape=label_shape, shuffle=mode == 'train')
    else:
      inputs = tf.placeholder(tf.float32, [batch_size, None,
                                           input_size]) 
    config['dropouts'] = [1.0 if mode == 'generate' else dropout for dropout in config['dropouts']]
    cell = get_deep_lstm(config['rnn_layers'], config['dropouts'])
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, final_state = tf.nn.dynamic_rnn(
        cell, inputs, sequence_length=lengths, initial_state=initial_state,
        swap_memory=True)

    outputs_flat = mg.common.flatten_maybe_padded_sequences(
            outputs, lengths)

    num_logits = num_classes
    logits_flat = tf.contrib.layers.linear(outputs_flat, num_logits)
    print(mode)
    if mode == 'train' or mode == 'eval':
      labels_flat = mg.common.flatten_maybe_padded_sequences(
              labels, lengths)
      softmax_cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=labels_flat, logits=logits_flat)
      predictions_flat = tf.argmax(logits_flat, axis=1)
      correct_predictions = tf.to_float(
          tf.equal(labels_flat, predictions_flat))
      event_positions = tf.to_float(tf.not_equal(labels_flat, default_event_label))
      no_event_positions = tf.to_float(tf.equal(labels_flat, default_event_label))

      loss = tf.reduce_mean(softmax_cross_entropy)
      optimizer = config['optimizer'](learning_rate=config['learning_rate'])
      train_op = tf.contrib.slim.learning.create_train_op(loss, optimizer)
      
      tf.summary.scalar('loss', loss)
      tf.add_to_collection('loss', loss)
      tf.add_to_collection('train_op', train_op)
      tf.add_to_collection('optimizer', optimizer)
    elif mode == 'generate':
      temperature = tf.placeholder(tf.float32, [])
      softmax_flat = tf.nn.softmax(
            tf.div(logits_flat, tf.fill([num_classes], temperature)))
      softmax = tf.reshape(
            softmax_flat, [batch_size, -1, num_classes])
    
      tf.add_to_collection('inputs', inputs)
      tf.add_to_collection('temperature', temperature)
      tf.add_to_collection('softmax', softmax)

      for state in tf_nest.flatten(initial_state):
        tf.add_to_collection('initial_state', state)
      for state in tf_nest.flatten(final_state):
        tf.add_to_collection('final_state', state)
  return build_graph

In [0]:
run_no = 0
runs_info = {}

In [0]:
runs_info[run_no] = {
    'rnn_layers': [512, 512],
    'dropouts': [0.70, 0.70],
    'optimizer': tf.train.RMSPropOptimizer,
    'learning_rate': 0.01,
    'encoder_decoder': mg.music.OneHotEventSequenceEncoderDecoder(
            mg.music.PerformanceOneHotEncoding(
                num_velocity_bins=32)),
    'steps': 4000
}

In [0]:
with tf.Graph().as_default():
  build_lstm(runs_info[run_no], 'train')()
  
  global_step = tf.train.get_or_create_global_step()
  loss = tf.get_collection('loss')[0]
  optimizer = tf.get_collection('optimizer')[0].minimize(loss)
  train_op = tf.get_collection('train_op')[0]
  logging_dict = {
      'global_step': global_step,
      'loss': loss
  }
  hooks = [
      tf.train.NanTensorHook(loss),
      tf.train.LoggingTensorHook(
          logging_dict, every_n_iter=100),
      tf.train.StopAtStepHook(runs_info[run_no]['steps'])
  ]
  
  tf.logging.info('Starting training loop...')
  tf.contrib.training.train(train_op=train_op, 
                            logdir='/content/log/run' + str(run_no),
                            hooks=hooks,
                            save_checkpoint_secs=1200,
                            save_summaries_steps=100)
  tf.logging.info('Training loop complete.')

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TFRecordDataset`.
INFO:tensorflow:Counting records in /

In [0]:
run_no += 1
run_no
runs_info

{0: {'dropouts': [0.2, 0.2],
  'encoder_decoder': <magenta.music.encoder_decoder.OneHotEventSequenceEncoderDecoder at 0x7f06741ede10>,
  'learning_rate': 0.01,
  'optimizer': tensorflow.python.training.adam.AdamOptimizer,
  'rnn_layers': [512, 512],
  'steps': 4000},
 1: {'dropouts': [0.2, 0.2],
  'encoder_decoder': <magenta.music.encoder_decoder.OneHotEventSequenceEncoderDecoder at 0x7f0674d08350>,
  'learning_rate': 0.01,
  'optimizer': tensorflow.python.training.adam.AdamOptimizer,
  'rnn_layers': [512, 512],
  'steps': 4000},
 2: {'dropouts': [0.2, 0.2],
  'encoder_decoder': <magenta.music.encoder_decoder.OneHotEventSequenceEncoderDecoder at 0x7f06739e2690>,
  'learning_rate': 0.001,
  'optimizer': tensorflow.python.training.adadelta.AdadeltaOptimizer,
  'rnn_layers': [512, 512],
  'steps': 4000},
 3: {'dropouts': [0.35, 0.35],
  'encoder_decoder': <magenta.music.encoder_decoder.OneHotEventSequenceEncoderDecoder at 0x7f0672010510>,
  'learning_rate': 0.009,
  'optimizer': tensorflo

### Generate, baby

In [0]:
# Upload a model
from google.colab import files
# files.upload()
files.upload()
files.upload()

Saving model.ckpt-1931.meta to model.ckpt-1931.meta


Saving model.ckpt-1931.index to model.ckpt-1931.index


{u'model.ckpt-1931.index': '\x00\x00\x06\x08\x01\x1a\x02\x08\x01\x00\x16\x11fully_connected/biases\x08\x01\x12\x05\x12\x03\x08\x84\x03(\x90\x0c5\xd3\x08\x8cc\x16\x08\x14/RMSProp\x08\x01\x12\x05\x12\x03\x08\x84\x03 \x90\x0c(\x90\x0c5\x01\x86\xa9\xa9\x1e\x02\x14_1\x08\x01\x12\x05\x12\x03\x08\x84\x03 \xa0\x18(\x90\x0c54\xa03\x19\x10\x07\x1aweights\x08\x01\x12\n\x12\x03\x08\x80\x04\x12\x03\x08\x84\x03 \xb0$(\x80\xc005Z\x0c\xac\x10\x17\x08\x1b/RMSProp\x08\x01\x12\n\x12\x03\x08\x80\x04\x12\x03\x08\x84\x03 \xb0\xe40(\x80\xc005\xa1\xfd\xa5K\x1f\x02\x1b_1\x08\x01\x12\n\x12\x03\x08\x80\x04\x12\x03\x08\x84\x03 \xb0\xa4a(\x80\xc005\xfbH=W\x00\x0b\x10global_step\x08\t\x12\x00 \xb0\xe4\x91\x01(\x085>d+\xd2\x00(\x16rnn/multi_rnn_cell/cell_0/lstm_cell/bias\x08\x01\x12\x05\x12\x03\x08\x80\x10 \xb8\xe4\x91\x01(\x80@5\xcae\x8d\x9d(\x08\x16/RMSProp\x08\x01\x12\x05\x12\x03\x08\x80\x10 \xb8\xa4\x92\x01(\x80@5TH\xce\x8f0\x02\x16_1\x08\x01\x12\x05\x12\x03\x08\x80\x10 \xb8\xe4\x92\x01(\x80@5T\xd8E\x06$\x06\x1d

In [0]:
!ls /content/log/run0

checkpoint
events.out.tfevents.1546873916.6a2b1eed8d88
graph.pbtxt
model.ckpt-0.data-00000-of-00001
model.ckpt-0.index
model.ckpt-0.meta
model.ckpt-1931.data-00000-of-00001
model.ckpt-1931.index
model.ckpt-1931.meta


In [0]:
!mkdir /content/log/run0

In [0]:
!cp  /content/model.ckpt-1931.index /content/log/run0

In [0]:
# Download files
from google.colab import files
number = 1931
run_no = 0
run_dir = '/content/log/run' + str(run_no) + '/'
files.download(run_dir + 'model.ckpt-' + str(number) + '.data-00000-of-00001')
files.download(run_dir + 'model.ckpt-' + str(number) + '.index')
files.download(run_dir + 'model.ckpt-' + str(number) + '.meta')

KeyboardInterrupt: ignored

In [0]:
runs_info

{0: {'dropouts': [1.0, 1.0, 1.0],
  'encoder_decoder': <magenta.music.encoder_decoder.OneHotEventSequenceEncoderDecoder at 0x7f3978f558d0>,
  'learning_rate': 0.001,
  'optimizer': tensorflow.python.training.gradient_descent.GradientDescentOptimizer,
  'rnn_layers': [512, 512, 512]},
 1: {'dropouts': [1.0, 1.0],
  'encoder_decoder': <magenta.music.encoder_decoder.OneHotEventSequenceEncoderDecoder at 0x7f39770f1b90>,
  'learning_rate': 0.001,
  'optimizer': tensorflow.python.training.gradient_descent.GradientDescentOptimizer,
  'rnn_layers': [512, 512]},
 2: {'dropouts': [0.2, 0.2],
  'encoder_decoder': <magenta.music.encoder_decoder.OneHotEventSequenceEncoderDecoder at 0x7f397761e050>,
  'learning_rate': 0.001,
  'optimizer': tensorflow.python.training.adam.AdamOptimizer,
  'rnn_layers': [512, 512]},
 3: {'dropouts': [0.2, 0.2],
  'encoder_decoder': <magenta.music.encoder_decoder.OneHotEventSequenceEncoderDecoder at 0x7f39774ca310>,
  'learning_rate': 0.01,
  'optimizer': tensorflow.py

In [0]:
import os
from magenta.protobuf import generator_pb2
from magenta.common import state_util
import tempfile
# Specify a trained model to use

def generate(checkpoint_dir, number, run_no):
  # Initialize the graph
  bundle_file = checkpoint_dir + 'bundle.mag'
  checkpoint_filename = os.path.join(checkpoint_dir, 'model.ckpt-' + str(number))
  metagraph_filename = os.path.join(checkpoint_dir, 'model.ckpt-' + str(number) + '.meta')
  print(checkpoint_filename)
  with tf.Graph().as_default() as g:
    
    # just checkpoint
    build_lstm(runs_info[run_no], 'generate')()
    sess = tf.Session(graph=g)
    saver = tf.train.Saver()
    tf.logging.info('Checkpoint used: %s', checkpoint_filename)
    saver.restore(sess, checkpoint_filename)
     
    try:
      tempdir = tempfile.mkdtemp()
      checkpoint_filename = os.path.join(tempdir, 'model.ckpt')
      saver = tf.train.Saver(sharded=False, write_version=tf.train.SaverDef.V1)
      saver.save(sess, checkpoint_filename, meta_graph_suffix='meta',
                 write_meta_graph=True)
      metagraph_filename = checkpoint_filename + '.meta'
      bundle = generator_pb2.GeneratorBundle()
      
      details = generator_pb2.GeneratorDetails(
                id='performance_with_dynamics',
                description='Performance RNN with dynamics (compact input)')
      bundle.generator_details.CopyFrom(details)

      with tf.gfile.Open(checkpoint_filename, 'rb') as f:
        bundle.checkpoint_file.append(f.read())
      with tf.gfile.Open(metagraph_filename, 'rb') as f:
        bundle.metagraph_file = f.read()

      with tf.gfile.Open(bundle_file, 'wb') as f:
        f.write(bundle.SerializeToString())   

    finally:
      if tempdir is not None:
        tf.gfile.DeleteRecursively(tempdir)
        

In [0]:
generate('/content/log/run0', 1931, 0)

/content/log/run0/model.ckpt-1931
generate
INFO:tensorflow:Checkpoint used: /content/log/run0/model.ckpt-1931
INFO:tensorflow:Restoring parameters from /content/log/run0/model.ckpt-1931


In [0]:
!python /usr/local/lib/python2.7/dist-packages/magenta/models/performance_rnn/performance_rnn_generate.py \
--bundle_file=/content/log/run0bundle.mag \
--output_dir=/content/log/ \
--config=performance_with_dynamics \
--num_outputs=1 \
--num_steps=1000 
# --primer_melody="[53, 57, 60, 57, 60, 57, 53, 57, 60, 57, 60, 57]"


/usr/local/lib/python2.7/dist-packages/scipy/signal/_max_len_seq.py:8: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._max_len_seq_inner import _max_len_seq_inner
/usr/local/lib/python2.7/dist-packages/scipy/signal/_upfirdn.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._upfirdn_apply import _output_len, _apply
/usr/local/lib/python2.7/dist-packages/scipy/signal/spectral.py:10: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._spectral import _lombscargle
/usr/local/lib/python2.7/dist-packages/scipy/signal/_peak_finding.py:13: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._peak_finding_utils import (_argmaxima1d, _select_by_peak_distance,
/usr/local/lib/python2.7/dist-packages/sklearn/linear_model/base.py:35: RuntimeWarning: numpy.dtype size ch

In [0]:
!ls -l /content/log/

total 16256
-rw-r--r-- 1 root root      236 Jan  7 13:14 2019-01-07_131159_1.mid
drwxr-xr-x 2 root root     4096 Jan  7 13:10 run0
-rw-r--r-- 1 root root 16635559 Jan  7 13:11 run0bundle.mag


In [0]:
!cp /content/log/2019-01-07_131159_1.mid /content

In [0]:
# from google.colab import files
files.download('/content/log/2019-01-07_131159_1.mid')

## VAE

In [0]:
from tensorflow.python.util import nest as tf_nest

config.latent_size
config.encoders[]
config.decoders[]

def build_vae(config, mode):
  
  def build_graph():

    encoder_decoder = config['encoder_decoder']
    input_size = encoder_decoder.input_size
    num_classes = encoder_decoder.num_classes
    default_event_label = encoder_decoder.default_event_label

    batch_size = 512
    label_shape = []
    learning_rate = config['learning_rate']
    inputs, labels, lengths = None, None, None
    if mode == 'train' or mode == 'eval':
      inputs, labels, lengths = mg.common.get_padded_batch(
                ['/content/sequences/training_performances.tfrecord'], batch_size, input_size,
                label_shape=label_shape, shuffle=mode == 'train')
    else:
      inputs = tf.placeholder(tf.float32, [batch_size, None,
                                           input_size])
    config['dropouts'] = [1.0 if mode == 'generate' else dropout for dropout in config['dropouts']]
    encoder_cell = get_deep_lstm(config['encoder_layers'], config['dropout'])          
    decoder_cell = get_deep_lstm(config['decoder_layers'], config['dropout'])
           
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, final_state = tf.nn.dynamic_rnn(
        cell, inputs, sequence_length=lengths, initial_state=initial_state,
        swap_memory=True)
    
    # Copmute our latent space
    mu = tf.layers.dense(outputs, config["z_size"], 
                         kernel_initializer=tf.random_normal_initializer(stddev=0.001))
    sigma = tf.layers.dense(outputs, config["z_size"], 
                            activation=tf.nn.softplus,
                            kernel_initializer=tf.random_normal_initializer(stddev=0.001))
    z = tfp.distributions.MultivariateNormalDiag(loc=mu, scale_diag=sigma)
    
      
    outputs_flat = mg.common.flatten_maybe_padded_sequencest(
            outputs, lengths)
    
    num_logits = num_classes
    logits_flat = tf.contrib.layers.linear(outputs_flat, num_logits)
                                           
    if mode == 'train' or mode == 'eval':
      labels_flat = mg.common.flatten_maybe_padded_sequences(
              labels, lengths)
      
      softmax_cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=labels_flat, logits=logits_flat)
      predictions_flat = tf.argmax(logits_flat, axis=1)
      correct_predictions = tf.to_float(
          tf.equal(labels_flat, predictions_flat))
      event_positions = tf.to_float(tf.not_equal(labels_flat, default_event_label))
      no_event_positions = tf.to_float(tf.equal(labels_flat, default_event_label))

      loss = tf.reduce_mean(softmax_cross_entropy)
      optimizer = config['optimizer'](learning_rate=config['learning_rate'])
      train_op = tf.contrib.slim.learning.create_train_op(loss, optimizer)
      
      tf.summary.scalar('loss', loss)
      tf.add_to_collection('loss', loss)
      tf.add_to_collection('train_op', train_op)
      tf.add_to_collection('optimizer', optimizer)
    elif mode == 'generate':
      temperature = tf.placeholder(tf.float32, [])
      softmax_flat = tf.nn.softmax(
            tf.div(logits_flat, tf.fill([num_classes], temperature)))
      softmax = tf.reshape(
            softmax_flat, [batch_size, -1, num_classes])
    
      tf.add_to_collection('inputs', inputs)
      tf.add_to_collection('temperature', temperature)
      tf.add_to_collection('softmax', softmax)

      for state in tf_nest.flatten(initial_state):
        tf.add_to_collection('initial_state', state)
      for state in tf_nest.flatten(final_state):
        tf.add_to_collection('final_state', state)
  return build_graph

# Old stuff

In [0]:
from tensorflow.keras.layers import LSTM, Dense, Input, Lambda, Dropout
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

def autoencoder(batch_size, features, label_dim, latent_dim=64):
  input_layer = Input(shape=(features.shape[1],1,))

  #Encoder layers
  hidden1 = LSTM(512)(input_layer)
  
  #Mean and std
  z_mean = Dense(latent_dim)(hidden1)
  z_log_sigma = Dense(latent_dim)(hidden1)
  
  #Sampled latent layer
  z = Lambda(sample)([z_mean, z_log_sigma])
  
#   hidden2 = LSTM(512, return_sequences=True)(z)
  
  #Decoder layers
#   hidden2 = RepeatVector(features.shape[1])(z)
#   hidden3 = LSTM(512, activation='sigmoid', return_sequences=True)(hidden2)
  
  #Output layers
#   output_layer = Dense(label_dim, activation='softmax')(hidden3)
  
  return Model(input_layer, z)
def sample(args):
  z_mean, z_log_var = args
  batch = K.shape(z_mean)[0]
  dim = K.int_shape(z_mean)[1]
  
  # by default, random_normal has mean=0 and std=1.0
  epsilon = K.random_normal(shape=(batch, dim))
  return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [0]:
def get_model(input_tensor):
  input_layer = Input(tensor=input_tensor)
  x = LSTM(512, return_sequences=True)(input_layer)
  x = Dropout(0.3)(x)
  x = LSTM(512)(x)
  x = Dense(512)(x)
  x = Dropout(0.3)(x)
  x = Dense(388, activation='softmax')(x)
  return Model(input_layer, x)
model = get_model(inputs)
model.summary()

NameError: ignored

### Data processing

In [0]:
import glob

def get_midi(notes, directory):
  for file in glob.glob(directory  + "/*.mid"):
    midi = converter.parse(file)
    notes_to_parse = None

    parts = instrument.partitionByInstrument(midi)
    if parts:
      notes_to_parse = parts.parts[0].recurse()
    else:
      notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
      if isinstance(element, note.Note):
        notes.append(str(element.pitch))
      elif isinstance(element, chord.Chord):
        notes.append('.'.join(str(n) for n in element.normalOrder))

  return notes

def get_input_output(notes, note_to_int, sequence_length):
  
  network_input, network_output = [], []
  for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
  return network_input, network_output

def normalize_data(network_input, network_output, sequence_length, n_vocab):
  n_patterns = len(network_input)
  normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
  normalized_input = normalized_input / float(n_vocab)
  normalized_output = keras.utils.to_categorical(network_output)
  
  return normalized_input, normalized_output

def get_data(notes, sequence_length=75):
  pitchnames = sorted(set(item for item in notes))
  note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
  n_vocab = len(set(notes))
  
  network_input, network_output = get_input_output(notes, note_to_int, sequence_length)
  normalized_input, normalized_output = normalize_data(network_input, 
                                                       network_output, 
                                                       sequence_length, 
                                                       n_vocab)
  
  return normalized_input, normalized_output, n_vocab, note_to_int, pitchnames

In [0]:
# Grab some data
!git clone https://github.com/Skuldur/Classical-Piano-Composer
# !git clone https://github.com/brannondorsey/midi-rnn
# !wget "www.kuhmann.com/Disklavier/Classical-II%20(215)%20MID.zip"
# !wget "www.kuhmann.com/Disklavier/Classical-I%20(001-215)%20MID.zip"

# Organise MIDI files for parsing
!mkdir data
# !unzip 'Classical-I (001-215) MID.zip' -q -d /content/data/
# !unzip 'Classical-II (215) MID.zip' -d /content/data/
!mv Classical-Piano-Composer/midi_songs/*.mid /content/data/
!ls data

Cloning into 'Classical-Piano-Composer'...
remote: Enumerating objects: 150, done.
remote: Total 150 (delta 0), reused 0 (delta 0), pack-reused 150
Receiving objects: 100% (150/150), 117.92 MiB | 10.86 MiB/s, done.
Resolving deltas: 100% (18/18), done.
 0fithos.mid
 8.mid
 ahead_on_our_way_piano.mid
 AT.mid
 balamb.mid
 bcm.mid
 BlueStone_LastDungeon.mid
 braska.mid
 caitsith.mid
 Cids.mid
 cosmo.mid
 costadsol.mid
 dayafter.mid
 decisive.mid
 dontbeafraid.mid
 DOS.mid
 electric_de_chocobo.mid
 Eternal_Harvest.mid
 EyesOnMePiano.mid
 ff11_awakening_piano.mid
 ff1battp.mid
'FF3_Battle_(Piano).mid'
'FF3_Third_Phase_Final_(Piano).mid'
 ff4-airship.mid
 Ff4-BattleLust.mid
 ff4-fight1.mid
 FF4.mid
 ff4pclov.mid
 ff4_piano_collections-main_theme.mid
 ff4-town.mid
 FF6epitaph_piano.mid
 ff6shap.mid
 Ff7-Cinco.mid
 Ff7-Jenova_Absolute.mid
 ff7-mainmidi.mid
 Ff7-One_Winged.mid
 ff7themep.mid
 ff8-lfp.mid
 FF8_Shuffle_or_boogie_pc.mid
 FFIII_Edgar_And_Sabin_Piano.mid
 FFIX_Piano.mid
 FFIXQuMarsh

### Training


In [0]:
notes = get_midi([], "/content/data")
network_input, network_output, n_vocab, note_to_int, pitchnames = get_data(notes)

In [0]:
n_vocab

358

In [0]:
model = autoencoder(network_input, n_vocab)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_27 (InputLayer)           (None, 75, 1)        0                                            
__________________________________________________________________________________________________
lstm_42 (LSTM)                  (None, 512)          1052672     input_27[0][0]                   
__________________________________________________________________________________________________
dense_61 (Dense)                (None, 64)           32832       lstm_42[0][0]                    
__________________________________________________________________________________________________
dense_62 (Dense)                (None, 64)           32832       lstm_42[0][0]                    
__________________________________________________________________________________________________
lambda_24 

### Noodling

In [0]:
from magenta.music import midi_io
def convert_midi(root_dir, sub_dir, full_file_path):
  """Converts a midi file to a sequence proto.
 
  Args:
    root_dir: A string specifying the root directory for the files being
        converted.
    sub_dir: The directory being converted currently.
    full_file_path: the full path to the file to convert.

  Returns:
    Either a NoteSequence proto or None if the file could not be converted.
  """
  try:
    sequence = midi_io.midi_to_sequence_proto(
        tf.gfile.FastGFile(full_file_path, 'rb').read())
  except midi_io.MIDIConversionError as e:
    tf.logging.warning(
        'Could not parse MIDI file %s. It will be skipped. Error was: %s',
        full_file_path, e)
    return None
  sequence.collection_name = os.path.basename(root_dir)
  sequence.filename = os.path.join(sub_dir, os.path.basename(full_file_path))
  sequence.id = note_sequence_io.generate_note_sequence_id(
      sequence.filename, sequence.collection_name, 'midi')
  tf.logging.info('Converted MIDI file %s.', full_file_path)
  return sequence



def convert_files(root_dir, sub_dir, writer, recursive=False):
  """Converts files.

  Args:
    root_dir: A string specifying a root directory.
    sub_dir: A string specifying a path to a directory under `root_dir` in which
        to convert contents.
    writer: A TFRecord writer
    recursive: A boolean specifying whether or not recursively convert files
        contained in subdirectories of the specified directory.

  Returns:
    A map from the resulting Futures to the file paths being converted.
  """
  dir_to_convert = os.path.join(root_dir, sub_dir)
  tf.logging.info("Converting files in '%s'.", dir_to_convert)
  files_in_dir = tf.gfile.ListDirectory(os.path.join(dir_to_convert))
  recurse_sub_dirs = []
  written_count = 0
  for file_in_dir in files_in_dir:
    tf.logging.log_every_n(tf.logging.INFO, '%d files converted.',
                           1000, written_count)
    full_file_path = os.path.join(dir_to_convert, file_in_dir)
    if (full_file_path.lower().endswith('.mid') or
        full_file_path.lower().endswith('.midi')):
      try:
        sequence = convert_midi(root_dir, sub_dir, full_file_path)
      except Exception as exc:  # pylint: disable=broad-except
        tf.logging.fatal('%r generated an exception: %s', full_file_path, exc)
        continue
      if sequence:
        writer.write(sequence)
    else:
      if recursive and tf.gfile.IsDirectory(full_file_path):
        recurse_sub_dirs.append(os.path.join(sub_dir, file_in_dir))
      else:
        tf.logging.warning(
            'Unable to find a converter for file %s', full_file_path)

  for recurse_sub_dir in recurse_sub_dirs:
    convert_files(root_dir, recurse_sub_dir, writer, recursive)

In [0]:
writer = tf.TFRecordWriter()

AttributeError: ignored